In [1]:
# Original source code by Yuwei Jiang @YuweiJ98, modfiied by David Johnson @djcomlab

In [2]:
import pandas as pd
import plotly.express as px
import plotly.offline as py
py.init_notebook_mode(connected=True)
import PyPDF2

In [3]:
large_paper_table = pd.read_excel('../data/sdg7-coding-auto.xlsx')
large_paper_table.shape

(399, 3)

In [4]:
# get all the paper titles
paper_titles = large_paper_table.title.unique()
title_list = paper_titles.tolist()

In [5]:
# convert all the expressions to abbreviations if they have abbreviations, i.e. United Nations -> UN
def convert_to_abbr(string):
    string = string.replace('Food and Agriculture organization of the United Nations','FAO').replace('FAOSTAT','FAO').replace('United Nations','UN').replace('Sustainable Development Solutions Network','SDSN').replace('United Nations Educational, Scientific and Cultural Organization','UNESCO').replace('Emissions Database for Global Atmospheric Research','EDGAR').replace('European Soil Data Center','ESDAC').replace('ES-DAC','ESDAC').replace('International Energy Agency','IEA').replace('Organisation for Economic Co-operation and Development','OECD').replace('International Monetary Fund','IMF').replace('Statistical, Economic, and Social Research and Training Center for Islamic Countries','SESRIC').replace('Asian Development Bank','ADB').replace('World Health Organization','WHO').replace('Global Administrative Areas','GADM').replace('Federation Federal State Statistics Service','Rosstat').replace('Department of Statistics Malaysia','DOSM').replace('Federal Institute for Geosciences and Natural Resources','BGR').replace('National Planning Commission','NPC').replace('National Environmental Protection Agency','ANPM').replace('Energy Information Administration','EIA').replace('Ministry of Domestic Trade Co-operatives and Consumerism','MDTCC').replace('National Oceanic and Atmospheric Administration','NOAA').replace('National Aeronautics and Space Administration','NASA').replace('sensing','sensor').replace('OpenStreetMap','OSM')
    return string

In [6]:
# create a list to store data resources for all the papers
resources_list=[]
for i in range(len(title_list)):
    # get all the datasource for a single paper
    paper_title = title_list[i]
    paper_df = large_paper_table[large_paper_table.title == paper_title]
    # 1. replace full names with abbr. for each cell in the column "data"
    data_list = paper_df['Data'].apply(convert_to_abbr).tolist()
    # 2. integrate the data column into a single string
    data_string = ','.join(data_list)
    # 3. split the string into a list, and remove all the duplicates
    data_list_abbr = data_string.split(',')
    data_list_abbr = list(dict.fromkeys(data_list_abbr))
    # 4. append
    resources_list.append(data_list_abbr)
    
resources_list

[['solar', 'electricity', 'survey', 'World Bank'],
 ['biomass', 'land', 'National Institute of Statistics', 'ANPM'],
 ['land', 'DOSM', 'MDTCC', 'OSM'],
 ['Eurostat', 'electricity', 'heat'],
 ['heat', 'electricity'],
 ['water', 'land', 'GIS'],
 ['electricity'],
 ['UN', 'solar'],
 ['Eurostat'],
 ['interview', 'survey', 'questionnaire'],
 ['Eurostat'],
 ['UN',
  'satellite imagery',
  'GIS',
  'solar',
  'electricity',
  'water',
  'land',
  'World Bank',
  'biomass'],
 ['water', 'survey', 'World Bank', 'IEA', 'ADB', 'UN', 'FAO', 'WHO'],
 ['Eurostat',
  'World Bank',
  'OECD',
  'IMF',
  'Global Carbon Atlas',
  'Rosstat',
  'land'],
 ['electricity',
  'survey',
  'World Bank',
  'sensor',
  'satellite imagery',
  'census',
  'GADM'],
 ['World Bank', 'land'],
 ['census', 'satellite imagery'],
 ['UN', 'SDSN', 'World Bank', 'OECD', 'survey', 'FAO', 'WHO'],
 ['survey', 'solar'],
 ['water', 'land', 'heat'],
 ['World Bank', 'sensor', 'electricity'],
 ['survey',
  'solar',
  'electricity',
  'i

In [7]:
def CountFrequency(my_list):
    # Creating an empty dictionary
    freq = {}
    for single_list in my_list:
        for item in single_list:
            if (item in freq):
                freq[item] += 1
            else:
                freq[item] = 1
    return freq

resource_freq = CountFrequency(resources_list)
for key, value in resource_freq.items():
    print (key,value)

solar 10
electricity 20
survey 17
World Bank 21
biomass 5
land 14
National Institute of Statistics 1
ANPM 1
DOSM 1
MDTCC 1
OSM 3
Eurostat 7
heat 3
water 10
GIS 5
UN 16
interview 3
questionnaire 3
satellite imagery 7
IEA 5
ADB 2
FAO 3
WHO 2
OECD 4
IMF 1
Global Carbon Atlas 1
Rosstat 1
sensor 3
census 6
GADM 2
SDSN 1
focus group 1
mineral 1
SESRIC 1
EIA 2
GPS 2
Freedom House 1
weather 1
UNESCO 1


In [8]:
resources = []
freq = []
for key, value in resource_freq.items():
    resources.append(key)
    freq.append(value)
data = {'resources':resources, 'freq':freq}
freq_df = pd.DataFrame(data)
freq_df

,resources,freq
0,solar,10
1,electricity,20
2,survey,17
3,World Bank,21
4,biomass,5
5,land,14
6,National Institute of Statistics,1
7,ANPM,1
8,DOSM,1
9,MDTCC,1


In [9]:
freq_df_whole = freq_df
characters = []
parent = []
frequency = []

In [10]:
def find_parent(string):
    parent = ''
    # for level 5
    if string == 'FAO' or string == 'UN' or string == 'SDSN' or string == 'UNESCO' or string == 'WHO':
        parent = 'UN organ.'
    if string == 'Eurostat' or string == 'EDGAR' or string == 'Copernicus' or string == 'ESDAC':
        parent = 'EU organ.'
    if string == 'World Bank' or string == 'IEA' or string == 'OECD' or string == 'IMF' or string == 'Global Carbon Atlas' or string == 'Global Footprint Network' or string == 'SESRIC' or string == 'BP Statistical Review of World Energy' or string == 'ADB' or string == 'World Pop' or string == 'GADM':
        parent = 'other I-organ.'
    if string == 'Rosstat' or string == 'General Statistics Office' or string == 'Central Statistical Office' or string == 'National Institute of Statistics' or string == 'DOSM' or string == 'National Bureau of Statistics of Tanzania':
        parent = 'statistics'
    if string == 'Ministry of Education and Science of Ukraine' or string == 'BGR' or string == 'NPC' or string == 'ANPM' or string == 'EIA' or string == 'MDTCC' or string == 'NOAA' or string == 'NASA' or string == 'Freedom House':
        parent = 'other N-organ.'
    # for level 4    
    if string == 'UN organ.' or string == 'EU organ.' or string == 'other I-organ.':
        parent = 'international'
    if string == 'statistics' or string == 'other N-organ.':
        parent = 'national'   
    # for level 3
    if string == 'questionnaire' or string == 'interview' or string == 'survey' or string == 'census' or string == 'focus group':
        parent = 'traditional statistics'
    if string == 'international' or string == 'national':
        parent = 'organ. and databases'
    if string == 'mineral' or string == 'water' or string == 'land' or string == 'electricity' or string == 'biomass' or string == 'solar' or string == 'heat':
        parent = 'resource'
    if string == 'satellite imagery' or string == 'GPS' or string == 'GIS' or string == 'OSM':
        parent = 'geographic'
    # for level 2
    if string == 'traditional statistics' or string == 'organ. and databases':
        parent = 'data sources'
    if string == 'weather' or string == 'sensor' or string == 'resource' or string == 'geographic':
        parent = 'data types'
    # for level 1
    if string == 'data sources' or string == 'data types':
        parent = ''
    return parent

In [11]:
def update_result(df, characters, parent, frequency, level):
    for i in range(len(df['resources'])):
        if df['resources'][i] in level:
            characters.append(df['resources'][i])
            frequency.append(df['freq'][i])
            iparent = find_parent(df['resources'][i])
            parent.append(iparent)
            # update the dataframe
            df.loc[i,'resources'] = iparent 

In [12]:
level5 = ['FAO','UN','SDSN','UNESCO','WHO',
          'Eurostat','EDGAR','Copernicus','ESDAC',
          'World Bank','IEA','OECD','IMF','Global Carbon Atlas','Global Footprint Network','SESRIC','BP Statistical Review of World Energy','ADB','World Pop','GADM',
          'Rosstat','General Statistics Office','Central Statistical Office','National Institute of Statistics','DOSM','National Bureau of Statistics of Tanzania',
          'Ministry of Education and Science of Ukraine','BGR','NPC','ANPM','EIA','MDTCC','NOAA','NASA','Freedom House']
level4 = ['UN organ.','EU organ.','other I-organ.',
          'statistics','other N-organ.']
level3 = ['questionnaire','interview','survey','census','focus group',
          'international','national',
          'mineral','water','land','electricity','biomass','solar','heat',
          'satellite imagery','GPS','GIS','OSM']
level2 = ['traditional statistics','organ. and databases',
          'weather','sensor','resource','geographic']
level1 = ['data sources','data types']
levels = [level5, level4, level3, level2, level1]

In [13]:
for i in range(len(levels)):
    update_result(freq_df_whole, characters, parent, frequency, levels[i])
    freq_df_whole = freq_df_whole.groupby('resources').sum().reset_index()

print(characters)
print(parent)
print(frequency)

['World Bank', 'National Institute of Statistics', 'ANPM', 'DOSM', 'MDTCC', 'Eurostat', 'UN', 'IEA', 'ADB', 'FAO', 'WHO', 'OECD', 'IMF', 'Global Carbon Atlas', 'Rosstat', 'GADM', 'SDSN', 'SESRIC', 'EIA', 'Freedom House', 'UNESCO', 'EU organ.', 'UN organ.', 'other I-organ.', 'other N-organ.', 'statistics', 'GIS', 'GPS', 'OSM', 'biomass', 'census', 'electricity', 'focus group', 'heat', 'international', 'interview', 'land', 'mineral', 'national', 'questionnaire', 'satellite imagery', 'solar', 'survey', 'water', 'geographic', 'organ. and databases', 'resource', 'sensor', 'traditional statistics', 'weather', 'data sources', 'data types']
['other I-organ.', 'statistics', 'other N-organ.', 'statistics', 'other N-organ.', 'EU organ.', 'UN organ.', 'other I-organ.', 'other I-organ.', 'UN organ.', 'UN organ.', 'other I-organ.', 'other I-organ.', 'other I-organ.', 'statistics', 'other I-organ.', 'UN organ.', 'other I-organ.', 'other N-organ.', 'other N-organ.', 'UN organ.', 'international', 'inte

In [14]:
fig = px.sunburst(
    names = characters,
    parents = parent,
    values = frequency,
    branchvalues = "total",
    color = parent,
    color_discrete_map={'data sources':'gold', 'data types':'darkblue'},
    width = 900,
    height = 900
)

fig.update_layout(
    margin = dict(t=100, l=10, r=10, b=10)
)

fig.update_traces(texttemplate = '%{label} %{percentRoot:.1%}')
fig.show()

In [15]:
fig.update_layout(margin_l=5, margin_t=5, margin_b=5, margin_r=5)
fig.write_image("../figures/sdg7-data-mapping.pdf")